In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import pandas as pd
from tqdm import tqdm

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode
from umap import UMAP
import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns

from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import numpy as np



d:\Arquivos_Acer\Documents\UFC\Introducao_ciencia_dados\env_topicos\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
df = pd.read_csv('./../../../datasets/fakeTelegram_SemTravaZap_BR_2022.csv')
df.head()

,Unnamed: 0,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,...,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres_especial_count
0,0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN,24
1,1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN,29
2,2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN,77
3,3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,8
4,4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0


In [3]:
ids_para_remover = ['b73c0b674b28d87375d78dcae2f1a21f','62b3aa83e4df122e48cb1f97583e403f',
                    '893874da7344daaa00b8b5b3dc295b59','38737f3c7a07586c4efbfa39aa345705',
                    'da4cbbc1b7b1192ad83bcdae613ac2c3', 'bf66a467459264e0d33e4c8518a82827',
                    '46e009035e6150d656d0c194db88fb07','811541f6ce49347d1798e8dd2d3cbf83',
                    '1d2a0a63333d12ad188e9e6f7f2e0419','7c379613d4da713fe959d8c7ddc11ce2',
                    'debe17fd10d504076d2df8682f63bfc6','e9713ae04a02a810d6f33dd956f42794',
                    'd6c77928db26721ce46aca2d549780f0','8e53464d3b01eea3e39d07e51ecbb1b4',
                    'bacd40da04dd7d13f646993bdcf8e79d','2273d1167a6212812d95dc8fadbae78e',
                    'cd3bb1bdf75be7595e6373171a5c2225','add8c1ba533c5e5450d92c061a5ee7bf',
                    'e198f90df1995528531dd43db0c935ea', '06dc9ac55ed64caab2bd97e9ab717302',
                    'c0110feb539d212836605b66192722dd','f71912700ac5331415408ce229681359',
                    '2ee692357a9c948351c43a9540e859ae', '25b66278176dabe814dfc25a405a2470',
                    '08b21cb1e7de74ef5fe1085230075523']

In [4]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,Unnamed: 0,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,...,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres_especial_count
0,0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN,24
1,1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN,29
2,2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,...,5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN,77
3,3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,8
4,4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,...,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557510,557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,...,5,2022-11-16 14:49:39.146502,0.1027,NaN,575796,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a,32
557511,557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,...,5,2022-11-16 14:49:39.847434,0.0000,NaN,1286443,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221,6
557512,557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,...,5,2022-11-16 14:49:39.922279,0.0000,NaN,13294,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,14
557513,557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,...,5,2022-11-16 14:49:39.992932,0.0000,NaN,1286444,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e,14


In [5]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



len(df_filtrado)

443705

In [6]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

In [8]:
df_geral.shape

(443705, 22)

In [9]:
df_geral.head()

,Unnamed: 0,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,...,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,caracteres_especial_count,text_processed
0,0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN,24,entao fato renato audio ouvi whatsapp ocorreu ...
1,1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,...,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN,29,saiu presidente 8 horas atras infelizmente con...
2,2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,...,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN,77,parte quase feita segundo turno completamos pa...
3,3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,...,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,8,achei seita maconarica
5,5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,...,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Texto,telegram,NaN,NaN,37,rindo agoraquem disse ia fazer acordo diabo pr...


## Modelagem de tópicos sem filtros

In [10]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_geral['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Batches: 100%|██████████| 13868/13868 [49:58<00:00,  4.62it/s] 
2025-07-13 09:29:49,879 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-13 15:40:41,422 - BERTopic - Dimensionality - Completed ✓
2025-07-13 15:40:41,467 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-13 15:41:36,352 - BERTopic - Cluster - Completed ✓
2025-07-13 15:41:36,448 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-07-13 15:41:46,374 - BERTopic - Representation - Completed ✓


In [11]:
# Exibir as top 10 palavras de um tópico específico
topic_model.get_topic(0)

# Exibir os 10 principais tópicos
for i in range(10):
    print(f"Tópico {i}:", topic_model.get_topic(i))


Tópico 0: [('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Tópico 1: [('community', 0.030758561187728187), ('electoral', 0.03074763103507868), ('court', 0.030708658426743907), ('decision', 0.030668232102886578), ('following', 0.030552214758924733), ('superior', 0.028032242688417197), ('brazil', 0.025234071692637583), ('tse', 0.019370202524232997), ('', 1e-05), ('', 1e-05)]
Tópico 2: [('urnas', 0.008701043628843923), ('votar', 0.007590137273252509), ('votos', 0.0059834257132915505), ('voto', 0.0051855204735236984), ('urna', 0.005115553993485259), ('impresso', 0.004759184998188638), ('votacao', 0.004586327278975714), ('fraude', 0.004565788646517224), ('votou', 0.0037936711673242115), ('votando', 0.0034329413672258137)]
Tópico 3: [('twitter', 0.39653322831434346), ('pjmedia', 0.0021968265643846252), ('ze', 0.0008327606670284094), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), (

In [ ]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/telegram/telegram_bertopic_sem_filtro.csv', index=False)

In [13]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
# Diversidade de tópicos
def topic_diversity(topics_words, top_n=10):
    words = [word for topic in topics_words for word in topic[:top_n]]
    unique_words = set(words)
    return len(unique_words) / (len(topics_words) * top_n)


# IRBO

def irbo(topics, topk=10, p=0.9):
    
    def rbo(list1, list2, p):
        overlap = 0.0
        rbo_score = 0.0
        depth = min(len(list1), len(list2))
        for d in range(1, depth + 1):
            if list1[d-1] in list2[:d] and list2[d-1] in list1[:d]:
                overlap += 1
            rbo_score += overlap / d * (p ** d)
        return (1 - p) * rbo_score

    scores = []
    for i in range(len(topics)):
        for j in range(i + 1, len(topics)):
            t1 = topics[i][:topk]
            t2 = topics[j][:topk]
            score = rbo(t1, t2, p)
            scores.append(score)

    return 1 - np.mean(scores) if scores else 0.0


In [15]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.5304
Coerência c_npmi:  nan
Coerência c_uci:   nan
Coerência u_mass:  nan
Diversidade:       0.6301
IRBO:              0.9998


## Com filtro religioso

In [16]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [17]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]


In [18]:
df_religiao.shape

(25762, 21)

In [19]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_religiao['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Batches: 100%|██████████| 806/806 [15:03<00:00,  1.12s/it]
2025-07-13 16:23:21,988 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-13 16:24:17,786 - BERTopic - Dimensionality - Completed ✓
2025-07-13 16:24:17,808 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-13 16:24:22,755 - BERTopic - Cluster - Completed ✓
2025-07-13 16:24:22,773 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-07-13 16:24:28,481 - BERTopic - Representation - Completed ✓


In [ ]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/telegram/telegram_bertopic_filtro_religioso.csv', index=False)

In [21]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


In [22]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.6927
Coerência c_npmi:  0.0818
Coerência c_uci:   -2.3859
Coerência u_mass:  -3.3507
Diversidade:       0.7603
IRBO:              0.9988


## Com filtro e religioso e exclusão de politico

In [23]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [24]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]

In [25]:
df_politico.shape

(8526, 21)

In [26]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_politico['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Batches: 100%|██████████| 267/267 [02:52<00:00,  1.55it/s]
2025-07-13 16:28:30,571 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-13 16:28:43,403 - BERTopic - Dimensionality - Completed ✓
2025-07-13 16:28:43,405 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-13 16:28:43,742 - BERTopic - Cluster - Completed ✓
2025-07-13 16:28:43,742 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-07-13 16:28:44,023 - BERTopic - Representation - Completed ✓


In [ ]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/telegram/telegram_bertopic_filtro_politico.csv', index=False)

In [28]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


In [29]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.6412
Coerência c_npmi:  0.1553
Coerência c_uci:   -1.8672
Coerência u_mass:  -5.3749
Diversidade:       0.7808
IRBO:              0.9959


In [30]:
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"deus, jesus, senhor, todos, povo, cristo, nome...","[0.0249, 0.0166, 0.0157, 0.0113, 0.0109, 0.010..."
1,1,"brasil, acima, todos, deus, vamos, povo, brasi...","[0.0677, 0.0274, 0.0246, 0.0212, 0.0196, 0.018..."
2,2,"video, assista, fiel, deus, videos, respondend...","[0.2712, 0.0839, 0.0823, 0.0603, 0.0562, 0.048..."
3,3,"unido, reino, tmealexeconomia, fonte, activetr...","[0.1151, 0.0989, 0.0818, 0.0646, 0.0511, 0.045..."
4,4,"lucro, obrigado, feliz, acabei, recebi, abenco...","[0.1219, 0.0952, 0.0847, 0.0713, 0.0682, 0.061..."
5,5,"derci, amo, video, acima, ramiro, coronel, ita...","[0.3534, 0.3177, 0.2438, 0.2236, 0.1951, 0.193..."
6,6,"managing, groups, welcome, tool, professional,...","[0.5018, 0.5018, 0.5018, 0.5018, 0.5018, 0.444..."
7,7,"esqueca, leia, salva, biblia, natural, nunca, ...","[0.2527, 0.2505, 0.2478, 0.2078, 0.195, 0.16, ..."
8,8,"vacina, covid19, vacinas, unido, vacinados, re...","[0.0996, 0.0576, 0.0542, 0.0454, 0.0445, 0.043..."
9,9,"foto, david, wilkerson, jesus, savior, valores...","[0.6552, 0.1877, 0.1327, 0.1269, 0.1062, 0.094..."
